In [7]:
import pandas as pd
import os
import re 

# List of CSV file paths for each model
csv_files = ["C:\\Users\\DELL ADMIN\\OneDrive - iitgn.ac.in\\Desktop\\table\\VGG_1block.csv", "C:\\Users\\DELL ADMIN\\OneDrive - iitgn.ac.in\\Desktop\\table\\VGG_3block.csv", "C:\\Users\\DELL ADMIN\\OneDrive - iitgn.ac.in\\Desktop\\table\\VGG_3block_data_aug.csv", "C:\\Users\\DELL ADMIN\\OneDrive - iitgn.ac.in\\Desktop\\table\\VGG16_finetuning_MLP.csv", "C:\\Users\\DELL ADMIN\\OneDrive - iitgn.ac.in\\Desktop\\table\\VGG_16_finetune_all.csv"]

# Initialize an empty list to store DataFrames
# Initialize an empty list to store DataFrames
dataframes = []

# Process each CSV file
for file in csv_files:
    # Read the CSV file
    df = pd.read_csv(file)
    
    # Extract the numerical value from the 'Training Loss' tensor string
    def extract_loss_value(loss):
        if isinstance(loss, str):
            # Match the number inside the tensor string (e.g., "tensor(0.6752, ...)")
            match = re.search(r"tensor\(([\d.]+)", loss)
            if match:
                return float(match.group(1))  # Extract the value inside tensor()
        return None  # Return None if no match or invalid data
    
    df['Training Loss'] = df['Training Loss'].apply(extract_loss_value)
    
    # Add a column for the model name (optional)
    model_name = os.path.splitext(os.path.basename(file))[0]
    df['Model'] = model_name  # Add a 'Model' column
    
    # Append the DataFrame to the list
    dataframes.append(df)

# Combine all DataFrames into one
combined_df = pd.concat(dataframes, ignore_index=True)

# Reorder columns to place 'Model' first (optional)
columns = ['Model'] + [col for col in combined_df.columns if col != 'Model']
combined_df = combined_df[columns]

# Save the combined DataFrame to a new CSV
combined_df.to_csv("combined_models.csv", index=False)

# Display the combined DataFrame
print(combined_df)

                  Model  Training Loss  Training Accuracy  \
0            VGG_1block         0.0122             93.125   
1            VGG_3block         3.9470            100.000   
2   VGG_3block_data_aug         0.3091             88.750   
3  VGG16_finetuning_MLP         7.7434            100.000   
4   VGG_16_finetune_all         0.6752             44.375   

   Number of Parameters  Training Time (seconds)  Testing Accuracy  
0              51381377               580.623758              80.0  
1              12938561               539.041337              80.0  
2              12938561               570.896288              85.0  
3             119549953              1563.240277             100.0  
4             134264641              1619.444218              50.0  


Question 1

Overall Observations:
Overfitting: Models with high training accuracy (100%) like VGG_3block and VGG16_finetuning_MLP show signs of overfitting, especially when training and testing accuracies diverge significantly.

Data Augmentation: VGG_3block_data_aug shows a good trade-off between training and testing performance, indicating that data augmentation has helped the model generalize better.

Complexity and Performance: More complex models, like VGG16_finetuning_MLP and VGG_16_finetune_all, with higher parameter counts, do not always result in better generalization. In fact, they can lead to overfitting or inefficient learning, as seen with the lower testing accuracies in VGG_16_finetune_all.

Training Time: Models with more parameters (like VGG16_finetuning_MLP and VGG_16_finetune_all) take much longer to train, which aligns with their complexity but doesn’t necessarily translate to better testing accuracy.

Conclusion:
Models with fewer parameters (e.g., VGG_3block) generally show better generalization, while more complex models (VGG16_finetuning_MLP and VGG_16_finetune_all) either overfit or require much longer training times without necessarily improving performance.
Data augmentation (as seen in VGG_3block_data_aug) appears to help the model generalize better and improve testing accuracy.

Question 2

Yes, data augmentation helps. It artificially increases the size of the training dataset by applying transformations (like rotations, flips, and zooms) to the images. This makes the model more robust and better at generalizing, reducing overfitting, especially when the dataset is small. For VGG16, it can help improve performance by exposing the model to more varied data, preventing it from memorizing the limited training images.

Question 3

Yes, the number of epochs matters when fine-tuning a model. If you fine-tune for too few epochs, the model may underfit and fail to learn useful features. If fine-tuned for too many epochs, the model may overfit the small dataset, especially for complex models like VGG16.

Comparison of models:
VGG (1 block): Fewer epochs might be enough as the model is simpler and less prone to overfitting.
VGG (3 blocks): Needs more epochs than VGG (1 block), but not too many to avoid overfitting.
VGG (3 blocks) with data augmentation: Data augmentation helps mitigate overfitting, so you can fine-tune for more epochs without risk.
Transfer learning (VGG16/VGG19, all layers): Fine-tuning for more epochs is crucial, but be cautious about overfitting due to the model's complexity.
Transfer learning (VGG16/VGG19, final layers): Fine-tuning the final layers requires fewer epochs, but still needs enough training to adapt to the new task.

Question 4

Yes, the model could be confused by images with similar features, poor quality, or rare poses. Overfitting can also cause confusion, as the model may struggle with images that differ from the training data. Data augmentation has helped by providing more varied examples. some images like a monkey cartoon and a half lion images are predicted wrongly.